# Clausius Rankine Cycle

## Introduction

- Aufbau des Cycles
- Definition Aufwand, Nutzen, Verlust

## Excercises 1 

- Variante 1: dissipativer Kondensator (Bilanzgrenze)
- Variante 2: Flusskühlung
- Ergebnisvergleich + Erklärung

## Excercise 2 (mit Variante 2)

- Wie wirken sich aus
  - Frischdampfdruck/-temperatur
  - Umgebungstemperatur
- Verbesserung des Kreislaufs
  - regenerative Speisewassererwärmung
  - ZÜ
  - (Kombination)
  - Wie kann man bei gleichen Frischdampfparametern mit der Exergieanalyse die Verbesserung begründen? Exergiezerstörung pro Komponente bei der Parametervariation